In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customerattritionprediction/Sample Submission.csv
/kaggle/input/customerattritionprediction/train.csv
/kaggle/input/customerattritionprediction/test.csv


In [5]:
pd.set_option('display.max_rows', None)

In [9]:
train = pd.read_csv('../input/customerattritionprediction/train.csv')
train.head()

,ID,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,SettlementProcess,QuarterlyPayment,GrandPayment,CustomerAttrition
0,B60LHTZLLPFU4IULNPUF,Female,No,No,Yes,48,Yes,Wifi Broadband,No,Yes,Yes,No,Check,96.259802,4628.129119,Yes
1,1RUJ7ONYZHYTWKOKO1SD,Male,No,Yes,No,45,Yes,Wifi Broadband,No,No,No,Yes,Bank,18.876470,922.085998,No
2,DZ9EEOQWSN0MYCBUPAM5,Male,Yes,Yes,No,59,Yes,Wifi Broadband,Yes,No,Yes,No,Card,106.016057,6205.891129,Yes
3,YZ43L1UQUQACTL1PXXZG,Male,No,No,No,14,Yes,Wifi Broadband,No,No,No,No,Check,80.108839,1248.655391,No
4,6P28152QGF60PDLSCB9M,Female,No,Yes,Yes,25,Yes,Wifi Broadband,No,Yes,No,Yes,Check,51.529773,1285.321277,No


In [10]:
vc = [col for col in train.columns if col not in ['ID', 'QuarterlyPayment', 'GrandPayment']]
for col in vc:
    print(train[col].value_counts())

Male      3213
Female    3125
Name: sex, dtype: int64
No     5309
Yes    1029
Name: Aged, dtype: int64
No     3273
Yes    3065
Name: Married, dtype: int64
No     4432
Yes    1906
Name: TotalDependents, dtype: int64
 2     287
 1     230
 0     207
 70    168
 3     157
-1     135
 71    132
 4     131
 5     126
 8     119
 72    119
 9     117
 69    116
 7     112
 6     107
 11     95
 67     94
 16     94
 13     91
 10     90
 68     90
 15     87
 18     85
 73     84
 64     82
 23     82
 12     79
 65     77
 14     75
 62     73
 22     72
 60     71
 24     71
 51     70
 61     70
 25     69
 34     69
 55     69
 66     68
 35     68
 56     67
 36     67
 53     67
 45     66
 20     66
 49     66
 17     66
 26     66
 54     65
 31     65
 21     64
 50     64
 42     63
 19     63
 48     62
 27     62
 29     62
 52     61
 40     61
 59     61
 47     59
 33     59
 46     58
 30     58
 57     58
 28     57
 43     56
 41     54
 63     54
 44     54
 37     52
 58 

In [11]:
train['MobileService'].replace({'Yes':1, 'No':0}, inplace=True)
train['TechnicalAssistance'].replace({'Yes':1, 'No':0}, inplace=True)
train['HardwareSupport'].replace({'Yes':1, 'No':0}, inplace=True)
train['CyberProtection'].replace({'Yes':1, 'No':0}, inplace=True)
train['FilmSubscription'].replace({'Yes':1, 'No':0}, inplace=True)
train['Aged'].replace({'Yes':1, 'No':0}, inplace=True)
train['Married'].replace({'Yes':1, 'No':0}, inplace=True)
train['TotalDependents'].replace({'Yes':1, 'No':0}, inplace=True)
train['sex'].replace({'Male':1, 'Female':0}, inplace=True)
train['4GService'].replace({'Wifi Broadband':2, 'Satellite Broadband':1, 'No':0}, inplace=True)
train['CustomerAttrition'].replace({'Yes':1, 'No':0}, inplace=True)
train = train[train['ServiceSpan']!=-2]
avg_ss = train['ServiceSpan'].mean()
df = pd.get_dummies(train, columns=['SettlementProcess'])

In [14]:
print(avg_ss)

31.95139655988638


In [15]:
train['ServiceSpan'].replace({-1:32}, inplace=True)
gr_mean = train['GrandPayment'].mean()
print(gr_mean)

2290.544206966606


In [16]:
df['ServiceSpan'].replace({-1:32}, inplace=True)
df['GrandPayment'].fillna(2291, inplace=True)

In [17]:
del train

In [18]:
df.dtypes

ID                               object
sex                               int64
Aged                              int64
Married                           int64
TotalDependents                   int64
ServiceSpan                       int64
MobileService                     int64
4GService                         int64
CyberProtection                   int64
HardwareSupport                   int64
TechnicalAssistance               int64
FilmSubscription                  int64
QuarterlyPayment                float64
GrandPayment                    float64
CustomerAttrition                 int64
SettlementProcess_Bank            uint8
SettlementProcess_Card            uint8
SettlementProcess_Check           uint8
SettlementProcess_Electronic      uint8
dtype: object

In [19]:
test = pd.read_csv('../input/customerattritionprediction/test.csv')
test['MobileService'].replace({'Yes':1, 'No':0}, inplace=True)
test['TechnicalAssistance'].replace({'Yes':1, 'No':0}, inplace=True)
test['HardwareSupport'].replace({'Yes':1, 'No':0}, inplace=True)
test['CyberProtection'].replace({'Yes':1, 'No':0}, inplace=True)
test['FilmSubscription'].replace({'Yes':1, 'No':0}, inplace=True)
test['Aged'].replace({'Yes':1, 'No':0}, inplace=True)
test['Married'].replace({'Yes':1, 'No':0}, inplace=True)
test['TotalDependents'].replace({'Yes':1, 'No':0}, inplace=True)
test['sex'].replace({'Male':1, 'Female':0}, inplace=True)
test['4GService'].replace({'Wifi Broadband':2, 'Satellite Broadband':1, 'No':0}, inplace=True)
test = pd.get_dummies(test, columns=['SettlementProcess'])

In [23]:
test['ServiceSpan'].replace({-1:32}, inplace=True)

In [25]:
test['GrandPayment'].fillna(2291, inplace=True)